In [19]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account

from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
import matplotlib.pyplot as plt
from decouple import config
import json


In [20]:
key_path = '../raw_data/skyscanner-insights-343713-06a72b2724ca.json'
key_path


'../raw_data/skyscanner-insights-343713-06a72b2724ca.json'

In [21]:
credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)


In [49]:
query = '''Select * from `skyscanner-insights-343713.raw_skyscanner_data_frankfurt.skyscanner_redirect_data_v5`
WHERE Timestamp BETWEEN DATETIME("2023-08-17")AND DATETIME_ADD("2023-08-18", INTERVAL 1 DAY)
AND CabinClass = 'ECONOMY'
AND Legscount > 1
AND OriginCity = 'SEL'
AND DestinationCity = 'OSA'
ORDER BY RAND()
'''


In [50]:
query_job = client.query(query)


In [51]:
results = query_job.result()
the_data = results.to_dataframe()


In [53]:
# the_data['OD'] = the_data['OriginCity'] + the_data['DestinationCity']
# the_data['OD'].value_counts()


In [54]:
keep_columns = ['OD', 'ItineraryRedirects', 'ODRedirects', "TravelHorizonDays" ]
data_2 = the_data[keep_columns]
data_2


KeyError: "['ItineraryRedirects', 'ODRedirects'] not in index"

In [48]:
the_data['ItinerarySortType'].value_counts(normalize=True)


ItinerarySortType
BEST                      0.551880
                          0.343760
CHEAPEST                  0.095435
FASTEST                   0.008060
DEPARTURE_OUTBOUND_ASC    0.000755
DEPARTURE_INBOUND_ASC     0.000110
Name: proportion, dtype: float64

In [28]:
the_data['CabinClass'].value_counts(normalize=True)


CabinClass
ECONOMY    1.0
Name: proportion, dtype: float64

In [29]:
colums_subset = ['Market', 'OriginAirport', 'DestinationAirport', 'Locale', 'CabinClass', 'Kind',
          'TripLengthNights', 'TravelType', 'TravelDistanceKm', 'TravelHorizonDays', 'RedirectType', 'ItineraryPosition', 'ItinerarySortType',
          'PriceCustomCurrencyPerPax',
          'Leg_0_StopsCount', 'Leg_1_StopsCount',
          'Leg_0_segment_0_OperatingCarrierIATA', 'Leg_0_segment_1_OperatingCarrierIATA', 'Leg_0_segment_2_OperatingCarrierIATA', 'Leg_0_segment_3_OperatingCarrierIATA',
          'Leg_1_segment_0_OperatingCarrierIATA', 'Leg_1_segment_1_OperatingCarrierIATA', 'Leg_1_segment_2_OperatingCarrierIATA', 'Leg_1_segment_3_OperatingCarrierIATA',
          'Leg_0_DurationMin', 'Leg_1_DurationMin'
]

all_columns = list(the_data.columns)


In [30]:
remaining_columns = list(set(all_columns) - set(colums_subset))
remaining_columns


['ThemeRomanceWeight',
 'Leg_1_segment_0_OriginName',
 'Leg_1_segment_2_FlightNumber',
 'Leg_0_segment_1_TravelDistanceKm',
 'UserRegionName',
 'Leg_0_segment_2_OriginIATA',
 'Leg_0_segment_3_TravelDistanceKm',
 'Leg_1_segment_3_MarketingCarrierIATA',
 'ThemeFoodWeight',
 'Device',
 'OSVersion',
 'Leg_1_segment_1_DestinationIATA',
 'LegsCount',
 'Leg_0_segment_3_MarketingCarrierIATA',
 'Leg_1_segment_3_DurationMin',
 'Leg_0_segment_3_OriginIATA',
 'Leg_1_segment_1_DepartureTimestamp',
 'Leg_0_segment_0_MarketingCarrierIATA',
 'Leg_0_segment_2_ArrivalTimestamp',
 'Leg_0_segment_1_DepartureTimestamp',
 'Leg_0_segment_2_DestinationName',
 'ThemeArtWeight',
 'Leg_0_segment_0_DurationMin',
 'Leg_1_segment_0_DurationMin',
 'Leg_1_segment_1_MarketingCarrierIATA',
 'Leg_1_segment_3_FlightNumber',
 'DepartureDate',
 'Leg_1_segment_3_OriginIATA',
 'Leg_1_segment_2_OriginIATA',
 'ProviderRank',
 'PassengerChild',
 'ThemeMountainWeight',
 'Leg_0_segment_3_DurationMin',
 'OriginCountry',
 'Leg_1_se

In [31]:
data_subset = the_data[colums_subset].copy()
data_subset


,Market,OriginAirport,DestinationAirport,Locale,CabinClass,Kind,TripLengthNights,TravelType,TravelDistanceKm,TravelHorizonDays,...,Leg_0_segment_0_OperatingCarrierIATA,Leg_0_segment_1_OperatingCarrierIATA,Leg_0_segment_2_OperatingCarrierIATA,Leg_0_segment_3_OperatingCarrierIATA,Leg_1_segment_0_OperatingCarrierIATA,Leg_1_segment_1_OperatingCarrierIATA,Leg_1_segment_2_OperatingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_0_DurationMin,Leg_1_DurationMin
0,ES,STN,AAR,es-ES,ECONOMY,RETURN,9,INTERNATIONAL,836,37,...,FR,,,,FR,,,,100,105
1,NG,PHC,ABV,en-GB,ECONOMY,RETURN,7,DOMESTIC,444,5,...,P4,,,,P4,,,,70,70
2,MX,MEX,ACA,es-MX,ECONOMY,RETURN,2,DOMESTIC,307,8,...,Y4,,,,Y4,,,,62,83
3,UK,LGW,ACC,en-GB,ECONOMY,RETURN,9,INTERNATIONAL,5065,75,...,BA,,,,BA,,,,395,400
4,UK,LHR,ACC,en-GB,ECONOMY,RETURN,21,INTERNATIONAL,5101,37,...,BA,,,,BA,,,,400,410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,NL,SFO,AMS,nl-NL,ECONOMY,RETURN,31,INTERNATIONAL,8786,60,...,KL,,,,KL,,,,615,660
196,KR,ICN,AMS,ko-KR,ECONOMY,RETURN,6,INTERNATIONAL,8552,37,...,KL,,,,KE,,,,810,715
197,NL,ICN,AMS,en-GB,ECONOMY,RETURN,89,INTERNATIONAL,8552,14,...,KE,,,,KE,,,,810,640
198,PL,KRK,AOI,pl-PL,ECONOMY,RETURN,4,INTERNATIONAL,868,26,...,RR,,,,RR,,,,100,100


In [32]:
data_subset['OD'] = data_subset['OriginAirport'] + data_subset['DestinationAirport']


In [33]:
data_subset[data_subset['CabinClass'] == 'ECONOMY']


,Market,OriginAirport,DestinationAirport,Locale,CabinClass,Kind,TripLengthNights,TravelType,TravelDistanceKm,TravelHorizonDays,...,Leg_0_segment_1_OperatingCarrierIATA,Leg_0_segment_2_OperatingCarrierIATA,Leg_0_segment_3_OperatingCarrierIATA,Leg_1_segment_0_OperatingCarrierIATA,Leg_1_segment_1_OperatingCarrierIATA,Leg_1_segment_2_OperatingCarrierIATA,Leg_1_segment_3_OperatingCarrierIATA,Leg_0_DurationMin,Leg_1_DurationMin,OD
0,ES,STN,AAR,es-ES,ECONOMY,RETURN,9,INTERNATIONAL,836,37,...,,,,FR,,,,100,105,STNAAR
1,NG,PHC,ABV,en-GB,ECONOMY,RETURN,7,DOMESTIC,444,5,...,,,,P4,,,,70,70,PHCABV
2,MX,MEX,ACA,es-MX,ECONOMY,RETURN,2,DOMESTIC,307,8,...,,,,Y4,,,,62,83,MEXACA
3,UK,LGW,ACC,en-GB,ECONOMY,RETURN,9,INTERNATIONAL,5065,75,...,,,,BA,,,,395,400,LGWACC
4,UK,LHR,ACC,en-GB,ECONOMY,RETURN,21,INTERNATIONAL,5101,37,...,,,,BA,,,,400,410,LHRACC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,NL,SFO,AMS,nl-NL,ECONOMY,RETURN,31,INTERNATIONAL,8786,60,...,,,,KL,,,,615,660,SFOAMS
196,KR,ICN,AMS,ko-KR,ECONOMY,RETURN,6,INTERNATIONAL,8552,37,...,,,,KE,,,,810,715,ICNAMS
197,NL,ICN,AMS,en-GB,ECONOMY,RETURN,89,INTERNATIONAL,8552,14,...,,,,KE,,,,810,640,ICNAMS
198,PL,KRK,AOI,pl-PL,ECONOMY,RETURN,4,INTERNATIONAL,868,26,...,,,,RR,,,,100,100,KRKAOI


In [34]:
data_subset['Leg_0_segment_0_OperatingCarrierIATA'].value_counts(normalize=True)[:30]


Leg_0_segment_0_OperatingCarrierIATA
FR    0.195
U2    0.150
KL    0.080
VY    0.070
      0.045
LS    0.025
XQ    0.020
RJ    0.020
PC    0.020
EI    0.020
EW    0.015
WA    0.015
RR    0.015
BA    0.015
5O    0.015
HV    0.015
W9    0.015
J9    0.010
QP    0.010
CX    0.010
W6    0.010
BY    0.010
AT    0.010
V7    0.010
I2    0.010
AR    0.010
SK    0.010
4C    0.005
FZ    0.005
FO    0.005
Name: proportion, dtype: float64